# OLS - Wooldridge Computer Exercise
## Chapter 6, Exercise 3

## To add a heading:
- Insert a new cell
- Type or paste-in content
- Place a single / just one "pound-sign" in front of the heading content
- Select "Markdown"
- Press "Shift", "Enter" at same time to convert to clean commentary

## To add a sub-heading:
- Insert a new cell
- Type or paste-in content
- Place two "pound-signs" in front of the sub-heading
- Select "Markdown"
- Press "Shift", "Enter" at same time to convert to clean commentary

## To add new bulleted documentation:

- Insert a new cell
- Type or paste-in content
- Place a "dash" character in front of the bulleted content
- Select "Markdown"
- Press "Shift", "Enter" at same time to convert to clean commentary

# References
- Wooldridge, J.M. (2016). Introductory econometrics: A modern approach (6thed.). Mason, OH: South-Western, Cengage Learning.
- Residual Plots: https://medium.com/@emredjan/emulating-r-regression-plots-in-python-43741952c034
- Understanding residual plots: https://data.library.virginia.edu/diagnostic-plots/
- VIF: https://etav.github.io/python/vif_factor_python.html
- VIF: https://en.wikipedia.org/wiki/Variance_inflation_factor
- Extracting various values from regression results: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html

# Instantiate libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms

from statsmodels.formula.api import ols
from statsmodels.compat import lzip
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.graphics.gofplots import ProbPlot

#import pandas.tseries.api as sm
#from tseries.formula.apt import ols

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr


plt.style.use('seaborn') # pretty matplotlib plots

plt.rc('font', size=14)
plt.rc('figure', titlesize=18)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=18)

# Latex markup language 
from IPython.display import Latex

# Data Read from csv

In [3]:
%%time
#df = pd.read_csv(BytesIO(csv_as_bytes),sep='|',nrows=100000)
df1 = pd.read_stata('C://Users//a1000391//Desktop//Machine Learning Lab//Pandas//python_seminar-master//WAGE2.dta')
print(df1.head())

   wage  hours   IQ  KWW  educ  exper  tenure  age  married  black  south  \
0   769     40   93   35    12     11       2   31        1      0      0   
1   808     50  119   41    18     11      16   37        1      0      0   
2   825     40  108   46    14     11       9   33        1      0      0   
3   650     40   96   32    12     13       7   32        1      0      0   
4   562     40   74   27    11     14       5   34        1      0      0   

   urban  sibs  brthord  meduc  feduc     lwage  
0      1     1      2.0    8.0    8.0  6.645091  
1      1     1      NaN   14.0   14.0  6.694562  
2      1     1      2.0   14.0   14.0  6.715384  
3      1     4      3.0   12.0   12.0  6.476973  
4      1    10      6.0    6.0   11.0  6.331502  
Wall time: 46.9 ms


In [4]:
df1['constant'] = 1

# Data Checks
- Columns

In [4]:
%%time
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 935 entries, 0 to 934
Data columns (total 18 columns):
wage        935 non-null int16
hours       935 non-null int8
IQ          935 non-null int16
KWW         935 non-null int8
educ        935 non-null int8
exper       935 non-null int8
tenure      935 non-null int8
age         935 non-null int8
married     935 non-null int8
black       935 non-null int8
south       935 non-null int8
urban       935 non-null int8
sibs        935 non-null int8
brthord     852 non-null float64
meduc       857 non-null float64
feduc       741 non-null float64
lwage       935 non-null float32
constant    935 non-null int64
dtypes: float32(1), float64(3), int16(2), int64(1), int8(11)
memory usage: 53.9 KB
Wall time: 22 ms


# Estimate: $log(wage) = \alpha + \beta_{1}educ + \beta_{2}exper + \beta_{3}(educ*exper) + \mu$

In [5]:
formula = '''lwage ~ educ + exper + (educ*exper)
'''
#model = ols(formula, df).fit(cov_type='HC0')
model = ols(formula, df1)
results = model.fit()
aov_table = statsmodels.stats.anova.anova_lm(results, typ=2)
print(aov_table)
print(results.summary())

                sum_sq     df           F        PR(>F)
educ         21.607355    1.0  140.376716  2.908162e-30
exper         5.539957    1.0   35.991493  2.834572e-09
educ:exper    0.675313    1.0    4.387312  3.647715e-02
Residual    143.303309  931.0         NaN           NaN
                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     48.41
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           4.47e-29
Time:                        16:44:19   Log-Likelihood:                -449.87
No. Observations:                 935   AIC:                             907.7
Df Residuals:                     931   BIC:                             927.1
Df Model:                           3                                         
Covarianc

## i - Holding experience fixed, return to another year of education:
## $ log(wage) = \beta_{1}\Delta educ + \beta_{3} (\Delta educ)exper$
## $ log(wage) / \Delta educ = (\beta_{1} + \beta_{3}exper)$


## ii - HypothesisTest:
## $ H_{o}: \beta_{3} = 0$
## $ H_{a}: \beta_{3} > 0$


In [6]:
tvalues = results.tvalues
tvalue_ed_ex = tvalues['educ:exper']
pvalues = results.pvalues
pvalue_ed_ex = pvalues['educ:exper']
print('iii - Running the hypothesis test we get:')
print('T-Stat of educ:exper')
print(tvalue_ed_ex)
print('Raw P-Value of educ:exper')
print(pvalue_ed_ex)
print('For a one-tailed test ==> P-Value of educ:exper / 2')
print(pvalue_ed_ex/2)

iii - Running the hypothesis test we get:
T-Stat of educ:exper
2.0945912148788666
Raw P-Value of educ:exper
0.03647714918816129
For a one-tailed test ==> P-Value of educ:exper / 2
0.018238574594080646


### iv - Return To Education When Experience = 10 Years
### Starting with:
### $ log(wage) = \beta_{0} + \beta_{1} educ + \beta_{2} exper + \beta_{3} educ*exper + \mu$

### Reparameterize (dropping in the mean as an "interesting value")
### $ log(wage) = \alpha_{0} + \theta_{1} educ + \theta_{2} exper + \beta_{3}(educ-mean(educ))(exper-mean(exper)) + \mu$

### In our case, the interesting value for experience is 10 years.
### First compute: $(exper-10)*educ$ and call it educexper10
### Next estimate: $ log(wage) = \beta_{0} + \Theta_{1} educ + \beta_{2} exper + \beta_{3}educexper10 + \mu$

In [7]:
#First compute educ_exper_10 = educ*(10*exper)
df1['educ_exper_10'] = df1['educ']*(df1['exper']-10)

#Then Estimate The Model
formula = '''lwage ~ educ + exper + educ_exper_10'''
#model = ols(formula, df).fit(cov_type='HC0')
model = ols(formula, df1)
results = model.fit()
aov_table = statsmodels.stats.anova.anova_lm(results, typ=2)
print(aov_table)
print(results.summary())

                   sum_sq     df           F        PR(>F)
educ            21.599793    1.0  139.857690  3.649794e-30
exper            1.868187    1.0   12.096430  5.285700e-04
educ_exper_10    0.193843    1.0    1.255122  2.628654e-01
Residual       143.784779  931.0         NaN           NaN
                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     47.21
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           2.11e-28
Time:                        16:44:53   Log-Likelihood:                -451.44
No. Observations:                 935   AIC:                             910.9
Df Residuals:                     931   BIC:                             930.2
Df Model:                           3                                    